In [1]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.1 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=146, type=traffic.traffic_light), Actor(id=145, type=traffic.traffic_light), Actor(id=144, type=traffic.traffic_light), Actor(id=143, type=traffic.traffic_light), Actor(id=142, type=traffic.traffic_light), Actor(id=141, type=traffic.traffic_light), Actor(id=140, type=traffic.traffic_light), Actor(id=139, type=traffic.traffic_light), Actor(id=138, type=traffic.traffic_light), Actor(id=137, type=traffic.traffic_light), Actor(id=136, type=traffic.traffic_light), Actor(id=135, type=traffic.traffic_light), Actor(id=134, type=traffic.traffic_light), Actor(id=133, type=traffic.traffic_light), Actor(id=132, type=traffic.traffic_light), Actor(id=131, type=traffic.traffic_light), Actor(id=130, type=traffic.traffic_light), Actor(id=129, type=traffic.traffic_light), Actor(id=128, type=traffic.traffic_light), Actor(id=127, type=traffic.traffic_light), Actor(id=126, type=traffic.traffic_light), Actor(id=125, type=traffic.traffic_light), Actor(id=124, type=traffic.traffic_light), Actor(id=1

In [3]:
manual_car = world.get_actor(181)
ego_vehicle = world.get_actor(181)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=-19.991625, y=255.057510, z=-0.000846)


In [5]:
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
df = initialize_dataframe()
#try:
while True:
    print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    if ego_on_highway:
        distance_to_junc = 80
    else:
        distance_to_junc = 20
    

    if junctions_detected and not ego_waypoint.is_junction:
        junctions_detected = False 
        same_junction = False
    elif junctions_detected:
        pass
    elif ego_waypoint.is_junction and on_junction == False:
        on_junction = True
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        junctions_detected = True
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction: #and  != get_junction_ahead(ego_waypoint, distance_to_junc).id:
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_id = junction.id
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)


    # CASES
    # 1. On city junction
    if ego_waypoint.is_junction and not highway_junction and junction.id != 459:
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction.id != 459:
        street_type = "Junction ahead"
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle
        )

        if junction_shape != "Error":
            print("Junction ahead")
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
            )
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")



    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id:
        street_type = "Highway with entry/exit"
        print("Highway Junction detected")
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            same_junction = True
            # print("selbst erkennnen")
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
            # # print grouped wps
            # for group in wps:
            #     print("######################### NEW GROUP #########################")
            #     for wp in group:
            #         print("-------------------")
            #         print(wp[0], wp[1])
            highway_shape = get_highway_shape(wps, junction)
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old
            wrong_shape = True

        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)            
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, direction_angle) 
                if not junction_old is None and junction_old.id != junction.id:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit, direction_angle) 
                
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")


    # 4. Highway entry / exit
    elif is_junction_ahead(ego_waypoint, 20)  and highway_junction and street_type != "Highway with entry/exit":
        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, True, on_entry, direction_angle) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")

    # 5. Normal Road
    else:
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)

    

    steering_angle = get_steering_angle(ego_vehicle)
    speed = get_speed(ego_vehicle)
    print("============================================================")
    if matrix:
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["steering_angle"] = steering_angle
        row_data["street_type"] = street_type
        # df = df.append(row_data, ignore_index=True)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()

    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
    vehicles_list = [ego_vehicle]
    print("\ndestroying %d vehicles" % len(vehicles_list))
    client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

    time.sleep(0.5)
"""

----------------------------------
Highway Junction detected
Junction object w/o shape
normal_highway 4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1

UnboundLocalError: local variable 'already_in_curve' referenced before assignment

In [18]:
junction_id_skip != junction.id

False

In [20]:
junction.id

255

In [19]:
junction_id_skip

255

In [13]:
is_junction_ahead(ego_waypoint, 80)

True

In [45]:
while True:
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    print(ego_waypoint.road_id)
    time.sleep(0.5)

22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
1594
1594
1594
1594
1594
1594
1594
1594
1594
1594
1594
1594
1594
1594


KeyboardInterrupt: 

In [86]:
# TEST: Print grouped wps in junction
for group in wps:
    for wp in group:
        
        if wp[0]:
            print(wp[0].__str__(), wp[1])
            print(wp[0].road_id)
            print(wp[0].lane_id)
            #print("yaw:", wp[0].get_forward_vector())
    print("----------------------------------------------------------------------------------------------------------------------")

Waypoint(Transform(Location(x=370.507843, y=-68.620674, z=0.004349), Rotation(pitch=0.000000, yaw=-180.726837, roll=0.000000))) end
749
-1
Waypoint(Transform(Location(x=370.552216, y=-65.120949, z=0.004349), Rotation(pitch=0.000000, yaw=-0.726851, roll=0.000000))) start
746
-1
----------------------------------------------------------------------------------------------------------------------
Waypoint(Transform(Location(x=381.045135, y=-77.793793, z=0.000000), Rotation(pitch=0.000000, yaw=-269.560883, roll=0.000000))) start
749
-1
Waypoint(Transform(Location(x=384.544983, y=-77.766968, z=0.000000), Rotation(pitch=0.000000, yaw=-269.560883, roll=0.000000))) start
760
-3
Waypoint(Transform(Location(x=388.044891, y=-77.740150, z=0.000000), Rotation(pitch=0.000000, yaw=-269.560883, roll=0.000000))) start
760
-2
Waypoint(Transform(Location(x=391.544800, y=-77.713326, z=0.000000), Rotation(pitch=0.000000, yaw=-269.560883, roll=0.000000))) start
760
-1
---------------------------------------

In [ ]:
# ToDo
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) config schreiben
 # 3) Doc strings (später)
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) exits
 # 2) Die ekligen junctions: Tankstelle und Highway Ampel (JP)
        # Einfach 4 Spure ballern
        # simpel wie anderen 
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) exitst
 # 2) Stadt Scenario testen 